In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import gc
import warnings
import os
import time

from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

#warnings.simplefilter(action='ignore', category=FutureWarning)

print(os.listdir("../Projet+Mise+en+prod+-+home-credit-default-risk"))

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))



['application_test.csv', '.DS_Store', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [3]:
# Training data
app_train = pd.read_csv('../my_csv_files/MY_df_for_pycaret.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (356251, 799)


,Unnamed: 0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,0,100002,1.0,0,0,0,0,202500.0,406597.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,100004,0.0,0,1,0,0,67500.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,100006,0.0,1,0,0,0,135000.0,312682.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,4,100007,0.0,0,0,0,0,121500.0,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import re

app_train = app_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train_df = app_train[app_train['TARGET'].notnull()]
test_df = app_train[app_train['TARGET'].isnull()]
#feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
feats = [f for f in train_df.columns if f not in ['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV',
                                                  'Unnamed: 0',
                                                  'index']]
train_df[feats]
train_df['TARGET']

0         1.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
307502    0.0
307503    0.0
307504    0.0
307505    1.0
307506    0.0
Name: TARGET, Length: 307507, dtype: float64

## Classification with PyCaret, with all features and all rows (no undersampling, no SMOTE)

In [4]:
#import classification module 
from pycaret.classification import * 

#intialize the setup (in Notebook env)
exp_clf = setup(train_df[feats], target = 'TARGET', fold_shuffle = True,
                train_size = 0.7, data_split_shuffle = True, session_id = 2)
                #imputation_type='iterative')

#intialize the setup (in Non-Notebook env)
#exp_clf = setup(train_df[feats], target = 'TARGET', html = False)

#intialize the setup (remote runs like Kaggle or GitHub actions)
#exp_clf = setup(train_df[feats], target = 'TARGET', silent = True)

,Description,Value
0,session_id,2
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(307507, 797)"
5,Missing Values,True
6,Numeric Features,563
7,Categorical Features,233
8,Ordinal Features,False
9,High Cardinality Features,False


In [5]:
compare_models(include = ["gbc", "lightgbm", "ada", "ridge", "svm", "nb"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9197,0.7720,0.0215,0.5566,0.0415,0.0357,0.0977,308.6330
lightgbm,Light Gradient Boosting Machine,0.9197,0.7811,0.0364,0.5402,0.0683,0.0587,0.1247,17.0920
ada,Ada Boost Classifier,0.9189,0.7612,0.0350,0.4683,0.0651,0.0545,0.1106,63.8040
ridge,Ridge Classifier,0.9187,0.0000,0.0024,0.1867,0.0046,0.0027,0.0100,15.5060
svm,SVM - Linear Kernel,0.8650,0.0000,0.0746,0.0971,0.0708,0.0089,0.0108,59.1130
nb,Naive Bayes,0.2178,0.5648,0.8766,0.0839,0.1532,0.0069,0.0274,3.7470


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
compare_models(include = ["ridge", "rf", "qda", "ada", "lda", "catboost"])

IntProgress(value=0, description='Processing: ', max=34)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:55:30
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Quadratic Discriminant Analysis


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9194,0.7145,0.0015,0.7600,0.0030,0.0027,0.0302,2652.0270
ridge,Ridge Classifier,0.9187,0.0000,0.0024,0.1867,0.0046,0.0027,0.0100,13.5720


In [7]:
compare_models(exclude = ["knn", "rf", "ada", "ridge", "qda", "catboost", "et", "lda"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9197,0.7720,0.0215,0.5566,0.0415,0.0357,0.0977,311.8660
lightgbm,Light Gradient Boosting Machine,0.9197,0.7811,0.0364,0.5402,0.0683,0.0587,0.1247,14.7730
xgboost,Extreme Gradient Boosting,0.9185,0.7720,0.0628,0.4614,0.1105,0.0930,0.1472,1575.4490
svm,SVM - Linear Kernel,0.8650,0.0000,0.0746,0.0971,0.0708,0.0089,0.0108,60.7150
dt,Decision Tree Classifier,0.8548,0.5424,0.1699,0.1492,0.1589,0.0798,0.0800,864.2130
nb,Naive Bayes,0.2178,0.5648,0.8766,0.0839,0.1532,0.0069,0.0274,4.4430
lr,Logistic Regression,0.0919,0.0586,0.0001,0.0100,0.0001,0.0000,0.0002,698.9290


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## Classification with PyCaret, with all features (SMOTE)

In [4]:
#import classification module 
from pycaret.classification import * 

#intialize the setup (in Notebook env)
exp_clf = setup(train_df[feats], target = 'TARGET', fold_shuffle = True,
                train_size = 0.7, data_split_shuffle=True, session_id = 2,
                fix_imbalance = True)
                #imputation_type='iterative')

#intialize the setup (in Non-Notebook env)
#exp_clf = setup(train_df[feats], target = 'TARGET', html = False)

#intialize the setup (remote runs like Kaggle or GitHub actions)
#exp_clf = setup(train_df[feats], target = 'TARGET', silent = True)

,Description,Value
0,session_id,2
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(307507, 797)"
5,Missing Values,True
6,Numeric Features,563
7,Categorical Features,233
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
del feats
del train_df
gc.collect()

In [ ]:
compare_models(include = ["lightgbm", "ridge", "svm", "nb"])  

In [ ]:
compare_models(include = ["ada", "ridge", "svm", "nb"])  

IntProgress(value=0, description='Processing: ', max=24)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:42:44
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Ridge Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9122,0.6970,0.0409,0.2364,0.0692,0.0471,0.0677,399.0210


In [ ]:
compare_models(include = ["gbc", "lightgbm", "ada", "ridge", "svm", "nb"])

IntProgress(value=0, description='Processing: ', max=34)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:14:21
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9192,0.7408,0.0062,0.4457,0.0123,0.0101,0.0447,1085.3580


## Classification with PyCaret, undersampling + selected variables

In [5]:
df_ft_selec_underbal = pd.read_csv("../my_csv_files/df_ft_selec_underbal.csv")

In [6]:
df_ft_selec_underbal

,Unnamed: 0,Unnamed: 0.1,INSTAL_AMT_INSTALMENT_MAX,PREV_NAME_YIELD_GROUP_high_MEAN,PREV_APP_CREDIT_PERC_MIN,PREV_NAME_CONTRACT_STATUS_Refused_MEAN,PREV_DAYS_DECISION_MEAN,AMT_CREDIT,INSTAL_PAYMENT_DIFF_MEAN,ANNUITY_INCOME_PERC,...,DEF_30_CNT_SOCIAL_CIRCLE,PREV_NAME_CONTRACT_STATUS_Approved_MEAN,PREV_NAME_YIELD_GROUP_low_action_MEAN,FLAG_DOCUMENT_3,NAME_FAMILY_STATUS_Married,REGION_RATING_CLIENT_W_CITY,INSTAL_AMT_PAYMENT_MIN,TARGET,SK_ID_CURR,index
0,54120,54120,370545.480,0.000000,0.716332,0.307692,-106.230769,590071.5,-2507.732609,0.138360,...,0.0,0.153846,0.000000,1,1,2,7461.675,0.0,162702,54122
1,188937,188937,26670.105,0.500000,0.949861,0.166667,-2121.166667,463284.0,709.625769,0.264320,...,0.0,0.666667,0.000000,1,1,2,98.055,0.0,319047,188940
2,74860,74860,8781.030,1.000000,0.949834,0.000000,-366.000000,284400.0,1254.432857,0.206867,...,0.0,1.000000,0.000000,1,1,3,3.060,0.0,186823,74862
3,5206,5206,16360.155,0.000000,1.000000,0.000000,-1612.500000,760225.5,1480.740000,0.359300,...,0.0,1.000000,0.500000,1,0,2,1.305,0.0,106092,5206
4,268000,268000,420489.990,0.166667,0.688383,0.083333,-561.333333,1331037.0,-5684.805000,0.343645,...,1.0,0.583333,0.000000,1,1,2,9.990,0.0,410534,268004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47644,307444,307444,129339.720,0.166667,0.863239,0.000000,-155.333333,450000.0,0.000000,0.158196,...,0.0,0.333333,0.000000,0,1,3,62.820,1.0,456186,307448
47645,307471,307471,938036.880,0.272727,0.759417,0.181818,-1557.454545,1303200.0,812.370609,0.325062,...,0.0,0.818182,0.090909,1,0,2,206.640,1.0,456215,307475
47646,307477,307477,148500.000,0.200000,0.860314,0.600000,-832.200000,297000.0,1628.805838,0.088780,...,0.0,0.400000,0.000000,1,1,2,0.000,1.0,456225,307481
47647,307485,307485,22500.000,0.000000,0.904478,0.000000,-307.000000,521280.0,0.000000,0.102620,...,1.0,1.000000,0.000000,1,0,2,24.300,1.0,456233,307489


In [9]:
import re

df_ft_selec = df_ft_selec_underbal
# Divide in training/validation and test data
df_ft_selec = df_ft_selec.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

train_df = df_ft_selec[df_ft_selec['TARGET'].notnull()]
test_df = df_ft_selec[df_ft_selec['TARGET'].isnull()]


# Cross validation model
#if stratified:
#    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
#else:
#    folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    
# Create arrays and dataframes to store results
feats = [f for f in train_df.columns if f not in ['Unnamed: 0', 'Unnamed0',
                                                  'Unnamed: 0.1',
                                                  'SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV',
                                                  'index']]

# Sklearn stuff
#from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Load and split dataset
#x, y = load_breast_cancer(return_X_y=True)

x_train, x_test, y_train, y_test = train_test_split(train_df[feats], train_df['TARGET'], 
                                                    test_size=0.3, random_state = 22)

In [10]:
#import classification module 
from pycaret.classification import * 

#intialize the setup (in Notebook env)
exp_clf = setup(train_df[feats], target = 'TARGET', fold_shuffle = True,
                train_size = 0.7, data_split_shuffle=True, session_id = 2)

,Description,Value
0,session_id,2
1,Target,TARGET
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(47649, 54)"
5,Missing Values,False
6,Numeric Features,49
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


In [11]:
compare_models(include = ["gbc", "lightgbm", "ada", "ridge", "svm", "nb"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6823,0.7473,0.6719,0.6806,0.6762,0.3643,0.3644,0.2770
gbc,Gradient Boosting Classifier,0.6816,0.7438,0.6631,0.6829,0.6728,0.3629,0.3630,6.1460
ridge,Ridge Classifier,0.6703,0.0000,0.6523,0.6710,0.6614,0.3403,0.3405,0.0460
ada,Ada Boost Classifier,0.6701,0.7300,0.6553,0.6696,0.6623,0.3400,0.3401,1.2270
svm,SVM - Linear Kernel,0.5404,0.0000,0.5194,0.5658,0.4731,0.0804,0.0933,0.2190
nb,Naive Bayes,0.5313,0.6029,0.1920,0.5971,0.2372,0.0547,0.0806,0.0270


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=2, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

## Classification with PyCaret, undersampling + selected variables + my metric

In [12]:
def my_comp_score(y_true, y_pred):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    tn_weight = 1
    fp_weight = 0
    fn_weight = -10
    tp_weight = 0

    # gain function for company, true positives and false positives don't matter that much to us
    # we want to penalise the false negatives, and we want to say yes to true negatives
    gain = tp*(tp_weight) + tn*(tn_weight) + fp*(fp_weight) + fn*(fn_weight)
    
    # best represents scenario where there are no false negatives or false postives
    # so all false positives would be correctly shown as negative
    # and all false negatives would be correctly show as positive
    best = (tn + fp) * tn_weight + (tp + fn) * tp_weight
    
    # baseline is a naive model that predicts non default(negative) for everyone
    # but all true positives and false negatives would be incorrectly shown as negative
    baseline = (tn + fp) * tn_weight + (tp + fn) * fn_weight
    
    score = (gain - baseline) / (best - baseline)
    
    return score

def my_comp_score_probs(y_true, y_pred):
    
    # for now y_pred is the probability that the value is 1
    y_use = [1 if i >= 0.5 else 0 for i in y_pred]
    
    y_use = pd.Series(y_use)

    tn, fp, fn, tp = confusion_matrix(y_true, y_use).ravel()
    
    tn_weight = 1
    fp_weight = 0
    fn_weight = -10
    tp_weight = 0

    # gain function for company, true positives and false positives don't matter that much to us
    # we want to penalise the false negatives, and we want to say yes to true negatives
    gain = tp*(tp_weight) + tn*(tn_weight) + fp*(fp_weight) + fn*(fn_weight)
    
    # best represents scenario where there are no false negatives or false postives
    # so all false positives would be correctly shown as negative
    # and all false negatives would be correctly show as positive
    best = (tn + fp) * tn_weight + (tp + fn) * tp_weight
    
    # baseline is a naive model that predicts non default(negative) for everyone
    # so all true positives and false negatives would be incorrectly shown as negative
    baseline = (tn + fp) * tn_weight + (tp + fn) * fn_weight
    
    score = ((gain - baseline) / (best - baseline))

    return score

In [13]:
# using my metric is not working, makes the scorer go null
add_metric("my_own_score", name = "My Custom Scorer", score_func = my_comp_score)

Name                                           My Custom Scorer
Display Name                                   My Custom Scorer
Score Function       <function my_comp_score at 0x7ff911060ee0>
Scorer                               make_scorer(my_comp_score)
Target                                                     pred
Args                                                         {}
Greater is Better                                          True
Multiclass                                                 True
Custom                                                     True
Name: my_own_score, dtype: object

In [18]:
remove_metric("My Custom Scorer")

In [19]:
# this are the same results as without the metric 
compare_models(include = ["gbc", "lightgbm", "ada", "ridge", "svm", "nb"])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6823,0.7473,0.6719,0.6806,0.6762,0.3643,0.3644,0.2890
gbc,Gradient Boosting Classifier,0.6816,0.7438,0.6631,0.6829,0.6728,0.3629,0.3630,6.7230
ridge,Ridge Classifier,0.6703,0.0000,0.6523,0.6710,0.6614,0.3403,0.3405,0.0460
ada,Ada Boost Classifier,0.6701,0.7300,0.6553,0.6696,0.6623,0.3400,0.3401,1.3170
svm,SVM - Linear Kernel,0.5404,0.0000,0.5194,0.5658,0.4731,0.0804,0.0933,0.2450
nb,Naive Bayes,0.5313,0.6029,0.1920,0.5971,0.2372,0.0547,0.0806,0.0220


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=2, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)